# AI PROJECT
## Training AI car to race in TORCS using neural networks


## Group Members:

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [47]:
#Load the dataset
df = pd.read_csv('combined_data.csv')
df.head()

,Angle,CurrentLapTime,Damage,DistanceFromStart,DistanceCovered,FuelLevel,Gear,LastLapTime,Opponent_1,Opponent_2,...,WheelSpinVelocity_1,WheelSpinVelocity_2,WheelSpinVelocity_3,WheelSpinVelocity_4,Z,Acceleration,Braking,Clutch,Gear,Steering
0,0.0,-0.982,0.0,6201.46,0.0,94.0000,0,0.0,16.9999,200.0,...,0.0,0.0,0.0,0.0,0.345263,1.0,0.0,0.64,1,-0.028559
1,0.0,-0.962,0.0,6201.46,0.0,94.0000,0,0.0,16.9999,200.0,...,0.0,0.0,0.0,0.0,0.345263,1.0,0.0,0.64,1,-0.028559
2,0.0,-0.942,0.0,6201.46,0.0,93.9999,0,0.0,16.9999,200.0,...,0.0,0.0,0.0,0.0,0.345263,1.0,0.0,0.64,1,-0.028559
3,0.0,-0.922,0.0,6201.46,0.0,93.9998,0,0.0,16.9999,200.0,...,0.0,0.0,0.0,0.0,0.345263,1.0,0.0,0.64,1,-0.028559
4,0.0,-0.902,0.0,6201.46,0.0,93.9997,0,0.0,16.9999,200.0,...,0.0,0.0,0.0,0.0,0.345263,1.0,0.0,0.64,1,-0.028559


In [48]:
# Strip trailing spaces from column names
df.columns = [col.strip() for col in df.columns]
df.columns.tolist()

['Angle',
 'CurrentLapTime',
 'Damage',
 'DistanceFromStart',
 'DistanceCovered',
 'FuelLevel',
 'Gear',
 'LastLapTime',
 'Opponent_1',
 'Opponent_2',
 'Opponent_3',
 'Opponent_4',
 'Opponent_5',
 'Opponent_6',
 'Opponent_7',
 'Opponent_8',
 'Opponent_9',
 'Opponent_10',
 'Opponent_11',
 'Opponent_12',
 'Opponent_13',
 'Opponent_14',
 'Opponent_15',
 'Opponent_16',
 'Opponent_17',
 'Opponent_18',
 'Opponent_19',
 'Opponent_20',
 'Opponent_21',
 'Opponent_22',
 'Opponent_23',
 'Opponent_24',
 'Opponent_25',
 'Opponent_26',
 'Opponent_27',
 'Opponent_28',
 'Opponent_29',
 'Opponent_30',
 'Opponent_31',
 'Opponent_32',
 'Opponent_33',
 'Opponent_34',
 'Opponent_35',
 'Opponent_36',
 'RacePosition',
 'RPM',
 'SpeedX',
 'SpeedY',
 'SpeedZ',
 'Track_1',
 'Track_2',
 'Track_3',
 'Track_4',
 'Track_5',
 'Track_6',
 'Track_7',
 'Track_8',
 'Track_9',
 'Track_10',
 'Track_11',
 'Track_12',
 'Track_13',
 'Track_14',
 'Track_15',
 'Track_16',
 'Track_17',
 'Track_18',
 'Track_19',
 'TrackPosition'

In [49]:
#Shape of DataFrame
df.shape

(176437, 79)

In [ ]:
# # Oversample race start data (3-5x more representation)
# race_start_data = df[df['CurrentLapTime'] < 10.0].copy()
# oversampled_starts = pd.concat([race_start_data] * 5)

# # Combine with regular data
# balanced_df = pd.concat([df, oversampled_starts])

# # Shuffle the data
# balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)

In [26]:
df = balanced_df
df.head()

,Angle,CurrentLapTime,Damage,DistanceFromStart,DistanceCovered,FuelLevel,Gear,LastLapTime,Opponent_1,Opponent_2,...,WheelSpinVelocity_1,WheelSpinVelocity_2,WheelSpinVelocity_3,WheelSpinVelocity_4,Z,Acceleration,Braking,Clutch,Gear,Steering
0,0.038905,36.206,3.0,1105.29,1220.250000,54.5903,4,0.000,200.0,200.0,...,121.45400,120.79500,121.47200,120.85200,0.321642,0.341759,0.0,0.000,4,0.008182
1,-0.043363,73.600,4391.0,1127.72,3003.650000,56.0194,3,125.706,200.0,200.0,...,88.37010,88.39190,88.67770,88.09270,0.344850,0.000000,0.0,0.000,3,-0.224291
2,0.034884,75.074,0.0,1172.78,1287.770000,49.9267,1,0.000,200.0,200.0,...,69.67710,69.27890,69.67940,69.28560,0.474639,1.000000,0.0,0.475,1,0.034809
3,0.001033,0.280,0.0,6201.81,0.351074,93.9729,1,0.000,200.0,200.0,...,7.90839,7.91359,9.80411,9.07698,0.345635,1.000000,0.0,0.640,1,-0.028148
4,-0.027877,6.230,0.0,4162.37,68.999000,49.9446,2,0.000,200.0,200.0,...,69.41360,69.30790,65.30790,65.38390,0.474598,1.000000,0.0,0.000,2,-0.019737


In [50]:
X = df.drop(columns=['Acceleration', 'Braking','Clutch', 'Gear', 'Steering'])
y = df[['Acceleration', 'Braking', 'Steering']]
X.head(), y.head()

(   Angle  CurrentLapTime  Damage  DistanceFromStart  DistanceCovered  \
 0    0.0          -0.982     0.0            6201.46              0.0   
 1    0.0          -0.962     0.0            6201.46              0.0   
 2    0.0          -0.942     0.0            6201.46              0.0   
 3    0.0          -0.922     0.0            6201.46              0.0   
 4    0.0          -0.902     0.0            6201.46              0.0   
 
    FuelLevel  LastLapTime  Opponent_1  Opponent_2  Opponent_3  ...  Track_16  \
 0    94.0000          0.0     16.9999       200.0       200.0  ...   13.9476   
 1    94.0000          0.0     16.9999       200.0       200.0  ...   13.9476   
 2    93.9999          0.0     16.9999       200.0       200.0  ...   13.9476   
 3    93.9998          0.0     16.9999       200.0       200.0  ...   13.9476   
 4    93.9997          0.0     16.9999       200.0       200.0  ...   13.9476   
 
    Track_17  Track_18  Track_19  TrackPosition  WheelSpinVelocity_1  \


In [51]:
y.shape

(176437, 3)

In [52]:
#Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled  

array([[-0.05706978, -1.30079194, -0.86052732, ..., -1.72669027,
        -1.72723789, -0.66799698],
       [-0.05706978, -1.3002641 , -0.86052732, ..., -1.72669027,
        -1.72723789, -0.66799698],
       [-0.05706978, -1.29973627, -0.86052732, ..., -1.72669027,
        -1.72723789, -0.66799698],
       ...,
       [-0.03102186,  0.76559071, -0.85920578, ...,  0.98542726,
         0.98485141, -0.85497871],
       [-0.03100488,  0.76611854, -0.85920578, ...,  0.98553849,
         0.98320581, -0.85678956],
       [-0.03102186,  0.76664638, -0.85920578, ...,  0.98682879,
         0.98494036, -0.85497097]])

In [53]:
#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((141149, 73), (35288, 73), (141149, 3), (35288, 3))

In [54]:
# Ytest is datafram so convert to array
y_test = y_test.values
print(y_test.shape)
print(y_test[0:5])


(35288, 3)
[[ 1.33041770e-03  0.00000000e+00 -7.60235712e-02]
 [ 0.00000000e+00  0.00000000e+00  3.57791744e-02]
 [ 1.00000000e+00  0.00000000e+00  6.30383438e-04]
 [ 0.00000000e+00  0.00000000e+00  1.83448792e-02]
 [ 1.00000000e+00  0.00000000e+00  2.44415686e-01]]


In [ ]:
#Drop 4th column of y_test
# y_test = np.delete(y_test, 3, axis=1)
# print(y_test.shape)
# print(y_test[0:5])


(35288, 5)
[[ 1.33041770e-03  0.00000000e+00  4.74999994e-01  1.00000000e+00
  -7.60235712e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.00000000e+00
   3.57791744e-02]
 [ 1.00000000e+00  0.00000000e+00  4.74999994e-01  1.00000000e+00
   6.30383438e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.00000000e+00
   1.83448792e-02]
 [ 1.00000000e+00  0.00000000e+00  0.00000000e+00  3.00000000e+00
   2.44415686e-01]]


In [ ]:
# y_train = np.delete(y_train, 3, axis=1)
# print(y_train.shape)
# print(y_train[0:5])

(141149, 5)
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  5.00000000e+00
  -3.24851903e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.00000000e+00
   7.43203908e-02]
 [ 1.00000000e+00  0.00000000e+00  4.74999994e-01  1.00000000e+00
   1.99891012e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.00000000e+00
   5.62625945e-01]
 [ 1.00000000e+00  0.00000000e+00  0.00000000e+00  2.00000000e+00
   2.01572012e-02]]


In [55]:
# Check X_train:
print(X_train[0:5])
print(X_train.shape)


[[-6.42613472e-02  1.24006485e+00 -8.60527324e-01  2.12521226e+00
   6.27126048e-01  2.17907905e+00 -7.47318180e-01  3.07024616e-01
   2.43482261e-01  2.22043998e-01 -3.05538603e+00  1.88709309e-01
   1.88346063e-01  1.90079032e-01  1.91869364e-01  1.77798093e-01
   1.82558963e-01  1.85880941e-01  1.93584994e-01  2.01260128e-01
   2.15439006e-01  2.40782543e-01  2.76913187e-01  2.76730765e-01
   5.01599323e-01  5.10772971e-01  2.21038372e-01  2.01451583e-01
   1.70278833e-01  1.59082392e-01  1.48145920e-01  1.40273600e-01
   1.57076070e-01  1.52929082e-01  1.53747211e-01  1.52462131e-01
   1.54361048e-01  1.80467933e-01  1.96740027e-01  1.92806191e-01
   2.29863338e-01  2.64481830e-01 -1.36753158e+00 -2.43758666e+00
   2.04049262e+00  3.29812923e+00 -5.60278161e-02  9.39463585e-02
   2.49091449e-03 -1.78736917e-02 -4.05482380e-02 -6.87274847e-02
  -5.53781703e-02 -2.13344690e-02 -3.93804485e-02  6.20705697e-02
   1.58156664e+00  1.56199128e+00  1.47447999e+00  3.58451108e-01
   1.42017

In [65]:
model = Sequential()
model.add(Dense(512, input_dim=73, activation='relu',kernel_initializer='he_normal'))
model.add(Dense(128, activation='relu',kernel_initializer='he_normal'))
model.add(Dense(64, activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(0.2))  # Dropout layer to prevent overfitting
model.add(Dense(32, activation='relu',kernel_initializer='he_normal'))
#Output layer with 3 neurons for 3 outputs
model.add(Dense(3, activation='linear',kernel_initializer='he_normal'))

model.compile(optimizer=Adam(learning_rate=0.007), loss='mean_squared_error', metrics=['mae'])


c:\Users\Zestro\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [66]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_25 (Dense)                │ (None, 512)            │        37,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 113,987 (445.26 KB)

 Trainable params: 113,987 (445.26 KB)

 Non-trainable params: 0 (0.00 B)

In [67]:
#Use EarlyStopping to prevent overfitting
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)


In [68]:
model.fit(X_train, y_train,validation_data=(X_test,y_test), epochs=100, batch_size=256, verbose=1,callbacks=[early_stopping])

Epoch 1/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.2590 - mae: 0.2088 - val_loss: 0.0227 - val_mae: 0.0798
Epoch 2/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0222 - mae: 0.0734 - val_loss: 0.0181 - val_mae: 0.0673
Epoch 3/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0194 - mae: 0.0656 - val_loss: 0.0163 - val_mae: 0.0565
Epoch 4/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0180 - mae: 0.0617 - val_loss: 0.0158 - val_mae: 0.0604
Epoch 5/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0174 - mae: 0.0597 - val_loss: 0.0151 - val_mae: 0.0622
Epoch 6/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0169 - mae: 0.0592 - val_loss: 0.0146 - val_mae: 0.0534
Epoch 7/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0155 - mae: 0.0567 - val_loss: 0.0142 - val_mae: 0.0524
Epoch 8/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0152 - mae: 0.0558 - val_loss: 0.0141 - val_mae: 0.0539
Epoch 9/100
552/552 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/

In [42]:
# 1. Identify race start samples in your dataframe
race_starts = df['CurrentLapTime'] < 10.0

# 2. Create a sample weights array
sample_weights = np.ones(len(df))
sample_weights[race_starts] = 5.0  # 5x importance for race starts

# 3. Split the weights along with your data
_, _, _, _, y_train_weights, y_test_weights = train_test_split(
    X_scaled, y, sample_weights, test_size=0.2, random_state=42)

# 4. Use standard MSE with sample_weights parameter
model.fit(X_train, y_train,
          validation_data=(X_test, y_test), 
          sample_weight=y_train_weights,
          epochs=100, 
          batch_size=256, 
          callbacks=[early_stopping],
          verbose=1)

Epoch 1/100
1039/1039 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0204 - mae: 0.0369 - val_loss: 0.0063 - val_mae: 0.0302
Epoch 2/100
1039/1039 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0152 - mae: 0.0301 - val_loss: 0.0063 - val_mae: 0.0283
Epoch 3/100
1039/1039 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0140 - mae: 0.0292 - val_loss: 0.0057 - val_mae: 0.0300
Epoch 4/100
1039/1039 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0135 - mae: 0.0291 - val_loss: 0.0082 - val_mae: 0.0426
Epoch 5/100
1039/1039 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0148 - mae: 0.0306 - val_loss: 0.0054 - val_mae: 0.0285
Epoch 6/100
1039/1039 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0140 - mae: 0.0293 - val_loss: 0.0062 - val_mae: 0.0290
Epoch 7/100
1039/1039 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0135 - mae: 0.0280 - val_loss: 0.0061 - val_mae: 0.0304
Epoch 8/100
1039/1039 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0135 - mae: 0.0294 - val_loss: 0.0055 - val_mae: 0.0293
Epoch 9/100
1039/1039 ━━━━━━━━━━

In [69]:
prediction = model.predict(X_test[10100].reshape(1, -1))
print("Predicted outputs for test data:")
print("Acceleration:", prediction[0][0])
print("Braking:", prediction[0][1])
print("Steering:", prediction[0][2])



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Predicted outputs for test data:
Acceleration: 0.06704831
Braking: 0.033972137
Steering: 0.18618459


In [70]:

print("Actual outputs for test data:")
print(y_test[10100])

Actual outputs for test data:
[0.        0.        0.2806612]


## Save the Model and Scaler

Now we'll save the trained model and the scaler so we can use them in the driver.py file. We'll use the newer Keras format for the model and joblib for the scaler.

In [71]:
import os
import joblib
from tensorflow.keras.models import load_model

# Create a models directory if it doesn't exist
model_dir = os.path.join(os.path.dirname(os.getcwd()), "models")
os.makedirs(model_dir, exist_ok=True)

# Save the model using the newer Keras format
model_path = os.path.join(model_dir, "torcs_driver_model.keras")
model.save(model_path)
print(f"Model saved to {model_path}")

# Save the scaler using joblib
scaler_path = os.path.join(model_dir, "torcs_scaler.joblib")
joblib.dump(scaler, scaler_path)
print(f"Scaler saved to {scaler_path}")

Model saved to c:\Users\Zestro\Desktop\AI\Project\TORCS-Using-NeuralNetworks\models\torcs_driver_model.keras
Scaler saved to c:\Users\Zestro\Desktop\AI\Project\TORCS-Using-NeuralNetworks\models\torcs_scaler.joblib


## Test Loading the Model and Scaler

Let's verify we can load the model and scaler correctly and get consistent predictions.

In [72]:
# Test loading the model and scaler
loaded_model = load_model(model_path)
loaded_scaler = joblib.load(scaler_path)

# Use the same sample_data as above

prediction_loaded = loaded_model.predict(X_test[10100].reshape(1,-1), verbose=0)

print("Prediction with loaded model:")
print("Acceleration:", prediction_loaded[0][0])
print("Braking:", prediction_loaded[0][1])
print("Steering:", prediction_loaded[0][2])


print("\nOriginal prediction:")
print("Acceleration:", prediction[0][0])
print("Braking:", prediction[0][1])
print("Steering:", prediction[0][2])


is_close = np.allclose(prediction_loaded, prediction, rtol=1e-5)
print(f"\nPredictions match: {is_close}")
if not is_close:
    print("\nDifference:")
    print(np.abs(prediction_loaded - prediction))

Prediction with loaded model:
Acceleration: 0.06704831
Braking: 0.033972137
Steering: 0.18618459

Original prediction:
Acceleration: 0.06704831
Braking: 0.033972137
Steering: 0.18618459

Predictions match: True


## Discussion on Gear Prediction

For gear selection, we have two options:

1. Use the neural network's prediction
2. Use rule-based gear selection based on RPM and speed

Gear selection is a discrete problem (values are integers: -1, 1, 2, 3, 4, 5, 6), while neural networks typically work better with continuous values. The current implementation in driver.py has specific rules for gear selection based on RPM thresholds, which is a common approach in racing.

For our implementation in driver.py, we'll use a hybrid approach:
- Use the neural network for acceleration, braking, clutch, and steering (continuous values)
- Use rule-based logic for gear selection (discrete values)

This approach combines the strengths of both methods for optimal performance.

## Sample Code for driver.py Integration

Here's a snippet showing how to implement the AI in driver.py:

In [ ]:
# Sample code for reference (not meant to be executed in notebook)
'''
def prepare_state_for_model(self):
    """Convert current car state to the format expected by the model"""
    # Create a numpy array with the same structure as the training data
    feature_array = np.array([
        [self.state.angle,
         self.state.curLapTime,
         self.state.damage,
         self.state.distFromStart,
         self.state.distRaced,
         self.state.fuel,
         self.state.gear,  # Current gear
         self.state.lastLapTime,
         # Add opponents (first 36 values)
         *[self.state.opponents[i] if i < len(self.state.opponents) else 200.0 for i in range(36)],
         self.state.racePos,
         self.state.rpm,
         self.state.speedX,
         self.state.speedY,
         self.state.speedZ,
         # Add track sensors (19 values)
         *self.state.track,
         self.state.trackPos,
         # Add wheel speeds (4 values)
         *self.state.wheelSpinVel,
         self.state.z
        ]]
    )
    
    # Create a DataFrame with the same column names as the training data
    with open(os.path.join(self.model_dir, "feature_names.pkl"), "rb") as f:
        feature_names = pickle.load(f)
    
    # Scale the feature array using the saved scaler
    scaled_features = self.scaler.transform(feature_array)
    return scaled_features
'''